In [ ]:
import sys
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
from sklearn.manifold import TSNE


offline.init_notebook_mode(connected=False)

sys.path.insert(0, '../../scripts/modeling_toolbox/')
# load the autoreload extension
%load_ext autoreload
# Set extension to reload modules every time before executing code
%autoreload 2

from metric_processor import MetricProcessor
import evaluation


# Source videos analysis

In [ ]:
path = '../../machine_learning/cloud_functions/data-temporal-large.csv'
data = pd.read_csv(path)
sources_df = data[data['path'] == data['title']]
sources_df = sources_df.loc[:, (sources_df != 0).any(axis=0)]
sources_df = sources_df.drop(['Unnamed: 0', 'attack', 'kind', 'path', 'dimension'], axis=1)
sources_df.head()

In [ ]:
non_series_sources_df = sources_df.filter(regex = '^((?!series).)*$').filter(regex = '^((?!ssim).)*$') 

print(non_series_sources_df.shape)
non_series_sources_df.head()

## Clustering algorithm analysis

In [ ]:
import time
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn import cluster, mixture
from sklearn.neighbors import kneighbors_graph
from itertools import cycle, islice

#X = np.asarray(non_series_sources_df.drop(['title', 'fps'], axis=1))
X = np.asarray(non_series_sources_df[['temporal_difference-mean', 
                                      'temporal_spatial_complexity-mean', 
                                      'temporal_difference-std', 
                                      'temporal_spatial_complexity-std',
                                      'temporal_difference-max', 
                                      'temporal_spatial_complexity-max'
                                      ]])
X = StandardScaler().fit_transform(X)

In [ ]:
# ============
# Create cluster object
# ============
gmm = mixture.GaussianMixture(
    n_components=5, covariance_type='full')

clustering_algorithms = [
    ('GaussianMixture', gmm)
]

# ============
# Create TSNE reduction
# ============
tsne = TSNE(n_components=2, verbose=1, perplexity=50, learning_rate=200, n_iter=2000)
tsne_results = tsne.fit_transform(X)

In [ ]:
offline.init_notebook_mode(connected=False)
gmm.fit(X)
traceSources = go.Scatter(
    x = non_series_sources_df['temporal_spatial_complexity-mean'],
    y =  non_series_sources_df['temporal_spatial_complexity-std'],
    mode = 'markers',
    text = non_series_sources_df['title'].values,
    showlegend = True,
     marker = dict(
         color = gmm.predict(X),
         size=non_series_sources_df['size']/(50000 * non_series_sources_df['fps']),
         line= dict(
                  color= 'rgb(255, 255, 255)',
                  width= 0
                 ),
        showscale = False,
        opacity = 0.8
    )
)
data = [traceSources]

layout = dict(title = 'Distribution of sources by means of temporal_spatial_complexity',
              hovermode= 'closest',
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False),
              showlegend= True,
             )

fig = dict(data=data, layout=layout)

offline.iplot(fig)

In [ ]:
offline.init_notebook_mode(connected=False)


traceTSNE = go.Scatter(
    x = tsne_results[:,0],
    y = tsne_results[:,1],
    mode = 'markers',
    text = non_series_sources_df['title'].values,
    showlegend = True,
     marker = dict(
         color = gmm.predict(X),
         size=non_series_sources_df['size']/(50000 * non_series_sources_df['fps']),
         line= dict(
                  color= 'rgb(255, 255, 255)',
                  width= 0
                 ),
        showscale = False,
        opacity = 0.8
    )
)
data = [traceTSNE]

layout = dict(title = 'TSNE (T-Distributed Stochastic Neighbour Embedding)',
              hovermode= 'closest',
              yaxis = dict(zeroline = False),
              xaxis = dict(zeroline = False),
              showlegend= True,
             )

fig = dict(data=data, layout=layout)

offline.iplot(fig)

In [ ]:
# Retrieve the dataset and attach the clustering information
features = ['temporal_dct-mean', 'temporal_gaussian-mean', 'size',
            'dimension', 'temporal_gaussian_difference-mean', 'temporal_difference-mean', 'title', 'path', 'attack_ID']


path = '../../machine_learning/cloud_functions/data-large.csv'

data = pd.read_csv(path)
print(data.shape)

def define_attack(row):
    attack_ID = row['path'].split('/')[-2]
    
    if '_' in attack_ID:
        return 1
    else:
        return 0
   

data['attack_ID'] = data.apply(lambda x: define_attack(x), axis=1)

attacks_df = data[data['attack_ID'] == 1]
sources_df = data[data['attack_ID'] == 0]
attacks_df = attacks_df.loc[:, (attacks_df != 0).any(axis=0)]

attacks_df = attacks_df[features]
sources_df = sources_df[features]

print('DATA:', data.shape)
display(data.head())
print('ATTACKS:', attacks_df.shape)
display(attacks_df.head())
print('SOURCES:', sources_df.shape)
display(sources_df.head())

In [ ]:
non_series_sources_df['gauss_cluster'] = gmm.predict(X)
non_series_sources_df['tsne_x'] = tsne_results[:,0]
non_series_sources_df['tsne_y'] = tsne_results[:,1]

display(non_series_sources_df.head())

def get_cluster(title):
    try:
        cluster_row = non_series_sources_df[non_series_sources_df['title']==title]
        
        return cluster_row[['gauss_cluster', 'tsne_x', 'tsne_y']].values[0]
    except:
        pass
    
cluster_df = pd.DataFrame(attacks_df.apply(lambda x: get_cluster(x['title']), axis=1))
cluster_df = cluster_df[0].apply(pd.Series)
attacks_df[['gauss_cluster', 'tsne_x', 'tsne_y']] = cluster_df

cluster_df = pd.DataFrame(sources_df.apply(lambda x: get_cluster(x['title']), axis=1))
cluster_df = cluster_df[0].apply(pd.Series)
sources_df[['gauss_cluster', 'tsne_x', 'tsne_y']] = cluster_df

In [ ]:
print('ATTACKS:', attacks_df.shape)
display(attacks_df.head())
print('SOURCES:', sources_df.shape)
display(sources_df.head())

In [ ]:
attacks_df = attacks_df.dropna(axis=0)
sources_df = sources_df.dropna(axis=0)
print('ATTACKS:', attacks_df.shape)
display(attacks_df.head())
print('SOURCES:', sources_df.shape)
display(sources_df.head())

train_prop=0.8
num_train = int(sources_df.shape[0]*train_prop)
df_train = sources_df[:num_train]
df_test = sources_df[num_train:]

df_train = df_train.sample(frac=1)
df_test = df_test.sample(frac=1)
df_attacks = attacks_df.sample(frac=1)

print('TRAIN:', df_train.shape)
display(df_train.head())
print('TEST:', df_test.shape)
display(df_test.head())
print('ATTACKS:', attacks_df.shape)
display(df_attacks.head())

x_train = df_train.drop(['title',
                         'path', 
                         'attack_ID'], axis=1)

x_test = df_test.drop(['title',
                       'path',
                       'attack_ID'], axis=1)


x_attacks = df_attacks.drop(['title',
                             'path',
                             'attack_ID'], axis=1)

x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
x_attacks = np.asarray(x_attacks)

# Scaling the data
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)
x_attacks = ss.transform(x_attacks)

In [ ]:
# Dataframe to store results
svm_results = pd.DataFrame(columns=['gamma', 'nu', 'n_components', 'TPR_test',
                                    'TNR', 'model', 'auc', 'f_beta', 'projection'])

# Train the models
svm_results = evaluation.one_class_svm(x_train, x_test, x_attacks, svm_results)



In [ ]:
svm_results.sort_values('f_beta', ascending=False).head()

In [ ]:
print(sources_df.shape)

In [ ]:
attacks_df['attack_ID'].unique()